## Transformación de datos

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [3]:
# Carga de datos
data_frame = pd.read_csv("1.7.3 Churn_Modelling.csv")
data_frame.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
# Valores nulos
for feature in data_frame.columns:
  print('Total de valores nulos de', feature, '=', data_frame[feature].isna().sum())

Total de valores nulos de RowNumber = 0
Total de valores nulos de CustomerId = 0
Total de valores nulos de Surname = 0
Total de valores nulos de CreditScore = 0
Total de valores nulos de Geography = 0
Total de valores nulos de Gender = 0
Total de valores nulos de Age = 0
Total de valores nulos de Tenure = 0
Total de valores nulos de Balance = 0
Total de valores nulos de NumOfProducts = 0
Total de valores nulos de HasCrCard = 0
Total de valores nulos de IsActiveMember = 0
Total de valores nulos de EstimatedSalary = 0
Total de valores nulos de Exited = 0


In [17]:
indice = data_frame.index
columnas_x=data_frame.columns[3:-1].tolist()
columnas_y=data_frame.columns[-1]

In [6]:
#Selecciona solo las características que se van a considerar en los posteriores análisis
X = data_frame.iloc[:, 3:-1].values
# La ùltima columna (sale o no sale) se separa del resto para analizar la␣
# relación que tiene ella con el resto
y = data_frame.iloc[:, -1].values

In [25]:
type(X)

pandas.core.frame.DataFrame

In [24]:
X=pd.DataFrame(X, columns= columnas_x) 

In [26]:
pd.DataFrame(X).head(10)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,0,42,2,0.0,1,1,1,101348.88
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58
2,502,France,0,42,8,159660.8,3,1,0,113931.57
3,699,France,0,39,1,0.0,2,0,0,93826.63
4,850,Spain,0,43,2,125510.82,1,1,1,79084.1
5,645,Spain,1,44,8,113755.78,2,1,0,149756.71
6,822,France,1,50,7,0.0,2,1,1,10062.8
7,376,Germany,0,29,4,115046.74,4,1,0,119346.88
8,501,France,1,44,4,142051.07,2,0,1,74940.5
9,684,France,1,27,2,134603.88,1,1,1,71725.73


In [8]:
pd.DataFrame(y).head(5)

,0
0,1
1,0
2,1
3,0
4,0


## Transformación de variables categóricas
Una variable categórica es aquella que toma valores desde un conjunto limitado de elementos.
Se revisarán 3 enfoques para tratar este tipo de variables. Antes de revisarlos es conveniente
preguntar ¿por qué deben ser tratadas?, la respuesta: los algoritmos que se revisarán más adelante
requieren, muchos de ellos, que las variables sean numéricas.
Los enfoques son:
1. Borrar las variables categóricas. Se aplica cuando la columna no aporta mayor valor al análisis.
2. Etiqueta codificada (Label Encoding(. Asigna a cada valor de la lista un número entero
diferente. Se debe tener cuidado porque el orden de los números no necesariamente representa
el orden de las categorías.
3. One-hot-encoding. Crea nuevas columnas indicando la presencia (o ausencia) de cada posible
valor en el set de datos original.

# Antes de comenzar a trabajar con nuestro set de datos, se revisará primero la forma en que trabaja
# el Label y el one-hot encoding

In [9]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
data = ['frio', 'frio', 'templado', 'frio', 'calor', 'calor', 'templado','frio', 'templado', 'calor']
values = array(data)
print('Valores:',values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print('Label encoder integer:',integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print('one-hot encoder:\n', onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

Valores: ['frio' 'frio' 'templado' 'frio' 'calor' 'calor' 'templado' 'frio'
 'templado' 'calor']
Label encoder integer: [1 1 2 1 0 0 2 1 2 0]
one-hot encoder:
 [[0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]
['frio']


## Analizando el resultado anterior
La matriz que aparece junto one-hot-encoder es de 3x3; esto
es porque se tienen tres valores distintos.
Si se revisa la primera fila: [0 1 0] significa que el primer valor corresponde a frío; es decir, la
primera columna representa el valor calor (codificado en 0), la segunda es frio (codificada en 1) y
la tercera columna es templado (codificada en 2)

In [28]:
#Se comienza con la columna que contiene el género
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X["Gender"] = le.fit_transform(X["Gender"])
pd.DataFrame(X)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,0,42,2,0.0,1,1,1,101348.88
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58
2,502,France,0,42,8,159660.8,3,1,0,113931.57
3,699,France,0,39,1,0.0,2,0,0,93826.63
4,850,Spain,0,43,2,125510.82,1,1,1,79084.1
...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.0,2,1,0,96270.64
9996,516,France,1,35,10,57369.61,1,1,1,101699.77
9997,709,France,0,36,7,0.0,1,0,1,42085.58
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52


In [ ]:
# Primero se revisan los valores únicos
pd.DataFrame(X)[1].unique()

# Se observan 3 valores distintos, por lo que se espera que se agreguen 3 columnas y que cada una de
# ellas contenga 0 (ausencia) o 1 (presencia) del valor original

In [ ]:
ct = ColumnTransformer(
...     [("text_preprocess", CountVectorizer(), "documents")

In [31]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), ["Geography"])],remainder='passthrough')
X = ct.fit_transform(X)
#X = np.array(ct.fit_transform(X))
#pd.DataFrame(X,index=indices, columns=columnas)

ValueError: Specifying the columns using strings is only supported for pandas DataFrames

In [ ]:
data_scaled = pd.DataFrame(scaled_features, index=df.index, columns=df.columns)

# **Comprobando la salida**
Se visualizan 3 columnas al inicio de la matriz; si se considera la primera fila es (1,0,0) y sabiendo
que el orden sería (France, Germany y Spain) indica que la primera observación sería de France.
El mismo análisis se hace para la segunda fila (0,0,1) que corresponde a Spain

# **Escalar las características**
Existen varias alternativas:
1. StandardScaler
2. MinMaxScaler

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
scaled_data = sc.fit_transform(X)
pd.DataFrame(scaled_data).head()

In [ ]:
scaler = MinMaxScaler()
scaled_data_1 = scaler.fit_transform(X)
pd.DataFrame(scaled_data_1).head()

In [ ]:
pd.DataFrame(scaled_data_1).describe()

In [ ]:
df_x = pd.DataFrame(scaled_data)
df_x.head()